In [ ]:
import CoolProp.CoolProp as CP
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from datetime import time
from scipy.optimize import minimize_scalar
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures
from datetime import time

In [ ]:
input_file = r"Input_time.xlsx"
data = pd.read_excel(input_file)
X = data[['T entrada lado caliente (ºC)', 'Tbh (ºC)', 'Presion de Alta' ,'T salida lado caliente (ºC)']]
y = data['T salida lado frio (ºC)']

In [ ]:
model_Tsalf = GradientBoostingRegressor(
n_estimators=300,
learning_rate=0.05,
max_depth=5,
min_samples_split=16,
min_samples_leaf=7,
max_features=None,
random_state=42
)
model_Tsalf.fit(X, y)

In [ ]:
X_entf= data[['Presion de Alta','Tbh (ºC)','T salida lado frio (ºC)','PotCompelec']]
y_entf = data['T entrada lado frio (ºC)']
model_Tentf = RandomForestRegressor(n_estimators=153,
max_depth=15,
min_samples_split=37,
min_samples_leaf=11,
random_state=42,
max_features='sqrt'
)
model_Tentf.fit(X_entf, y_entf)

In [ ]:
def prediction_Tsalf(input_data):
    predictions = model_Tsalf.predict(input_data)
    return predictions

def prediction_Tentf(input_data):
    predictions = model_Tentf.predict(input_data)
    return predictions

In [ ]:
def f_Qevap(Pcond,Psuc,Qevap):
    p1 = Pcond
    p2 = Pcond
    p3 = Psuc
    p4 = Psuc

    # CAMBIAR UNIDADES - Presion en Pa
    def barg_to_pa(barg):
        return (barg + 1) * 100000
    p1 = barg_to_pa(p1)
    p2 = barg_to_pa(p2)
    p3 = barg_to_pa(p3)
    p4 = barg_to_pa(p4)

    # Defino eficiencia isentropica compresor a partir de datos historico
    ef = 0.67
    #Funcion para calcular entalpia x = 0
    def calculate_enthalpy_ls(p):
        return CP.PropsSI('H', 'P', p, 'Q', 0, 'Ammonia')
    
    #Funcion para calcular entalpia x = 1
    def calculate_enthalpy_gs(p):
        return CP.PropsSI('H', 'P', p, 'Q', 1, 'Ammonia')
    #Funcion para calcular entropia, x=1
    def calculate_entropy_gs(p):
        return CP.PropsSI('S', 'P', p, 'Q', 1, 'Ammonia')
    #Funcion para calcular entalpia, con P y s
    def calculate_enthalpy_ps(p,s):
        return CP.PropsSI('H', 'P', p, 'S', s, 'Ammonia')
    def calculate_temperature(p):
        return CP.PropsSI('T', 'P', p, 'Q', 1, 'Ammonia')
    def calculate_enthalpy_temp(p,t):
        return CP.PropsSI('H', 'P', p, 'T', t, 'Ammonia')
    def calculate_entropy_temp(p,t):
        return CP.PropsSI('S', 'P', p, 'T', t, 'Ammonia')
    

    # Determino punto 2 - liquido saturado
    h2 = calculate_enthalpy_ls(p2)
    h3 = h2
    #Determino punto 4 - vapor saturado
    superheat=5.6
    t4=calculate_temperature(p4)
    t4=t4+superheat
    h4 = calculate_enthalpy_temp(p4,t4)
    s4 = calculate_entropy_temp(p4, t4)
    #Determino punto s1s
    s1s = s4
    h1s = calculate_enthalpy_ps(p1,s1s)
    # Determino punto 1 a partir de eficiencia
    h1 = (h1s-h4)/ef + h4
    #Determino caudal a partir de Qevap
    m = Qevap*1000/(h4- h3)

    #Determino Q Evap del compresor y Qcond (en kW)
    PotComp =( m*(h1-h4)/1000)
    PotComp_elec=PotComp/0.75
    Qcond = m*(h1 - h2)/1000
    return PotComp_elec,Qcond
        

In [ ]:
def calculo_costos(hora,PotComp, PotBomb, Qfueloil):
    # Paso los requerimientos a kWh
    PotComp_kWh = PotComp
    PotBomb_kWh = PotBomb
    Qfueloil_kWh = Qfueloil

    # Tarifa Conaprole - CG3
    llano_min1 = time(7)
    llano_max1 = time(18)
    llano_min2 = time(22)
    llano_max2 = time(0)
    punta_max = time(22)
    punta_min = time(18)
    valle_min = time(0)
    valle_max = time(7)
    precio_llano = 3.664
    precio_valle = 2.230
    precio_punta = 5.025
    precio_fueloil = 0.49 # USD / kg
    PCfueloil = 9600 # kcal/kg
    rend = 0.82 # Rendimiento del combustible
    costo_kcal = precio_fueloil/(PCfueloil*rend) # USD/kcal
    #Paso a costo KJ
    costo_kJ = costo_kcal/4.18 # USD/kJ
    costo_kWh = 40*costo_kJ*3600 #$/kWh
    costo_fueloil = Qfueloil_kWh*costo_kWh
    costo_bba=0
    costo_comp=0

    if llano_min1 <= hora < llano_max1: # de 7 a 18
        costo_bba = precio_llano*PotBomb_kWh
        costo_comp = precio_llano*PotComp_kWh
    if punta_min <= hora < punta_max: # de 18 a 22
        costo_bba = precio_punta*PotBomb_kWh
        costo_comp = precio_punta*PotComp_kWh
    if llano_min2 <= hora: #< llano_max2: # de 22 a 24
        costo_bba = precio_llano*PotBomb_kWh
        costo_comp = precio_llano*PotComp_kWh
    if valle_min <= hora < valle_max: # de 00 a 7
        costo_bba = precio_valle*PotBomb_kWh
        costo_comp = precio_valle*PotComp_kWh
    return costo_fueloil, costo_bba, costo_comp


In [ ]:
def calculo_precios(hora):
    llano_min1 = time(7)
    llano_max1 = time(18)
    llano_min2 = time(22)
    llano_max2 = time(0)
    punta_max = time(22)
    punta_min = time(18)
    valle_min = time(0)
    valle_max = time(7)
    precio_llano = 3.664
    precio_valle = 2.230
    precio_punta = 5.025
    if llano_min1 <= hora < llano_max1: # de 7 a 18
        precio=precio_llano
        return precio

    if punta_min <= hora < punta_max: # de 18 a 22
        precio=precio_punta
        return precio
    
    if llano_min2 <= hora: #< llano_max2: # de 22 a 24
        precio=precio_llano
        return precio
    
    if valle_min <= hora < valle_max: # de 00 a 7
        precio=precio_valle
        return precio
    
    return precio

In [ ]:
def f_dinamica_bomba(Pcond,PotComp_elec,Tbh,caudal_cold, caudal_hot,Qcond_bomba):
    deltat_caliente = Qcond_bomba/(4.18*caudal_hot)
    Tsal_cal = 70
    Tent_cal = Tsal_cal - deltat_caliente
    pred_Tsalf = pd.DataFrame([[Tent_cal, Tbh, Pcond, Tsal_cal]], columns=['Tentrada lado caliente (ºC)', 'Tbh (ºC)', 'Presion de Alta' ,'T salida lado caliente (ºC)'])
    Tsal_fria = prediction_Tsalf(pred_Tsalf)
    pred_Tentf = pd.DataFrame([[Pcond, Tbh, Tsal_fria, PotComp_elec]],
    columns=['Presion de Alta','Tbh (ºC)','T salida lado frio (ºC)','PotComp elec'])
    Tent_fria = prediction_Tentf(pred_Tentf)
    delta_frio = Tent_fria - Tsal_fria
    Qevap_bomba = caudal_cold*4.18*delta_frio
    PotBomba = Qcond_bomba-Qevap_bomba
    
    return Qevap_bomba,PotBomba,Tent_fria,Tsal_fria,Tent_cal, Tsal_cal

In [ ]:
def Calculo_de_potencias_de_bomba_y_compresor(Pcond,Pevap,Qevap,Tbh,caudal_cold,caudal_pozo,Tservicios,Tpozo, caudal_hot, hora):
    Qreq = caudal_pozo*4.18*(Tservicios-Tpozo)
    ratio_bomba = 1
    ratio_caldera = 0
    costo = 0
    ratio_bomba_vector = []
    ratio_caldera_vector = []
    costos = []
    Potencia_bomba_vec = []
    Qevap_bomba_vec = []
    Qevap_bomba = 0
    Qcondbomba = 0
    PotBomba = 0
    COP_bba = 0
    (PotComp_elec,Qcond) = f_Qevap(Pcond,Pevap,Qevap)
    if Qreq <=400:
        Qreqm = Qreq
        Qex = 0
    else:
        Qreqm = 400
        Qex = Qreq - Qreqm
    
    precio_por_kWh_fueloil = 2.144357567977594

    while ratio_bomba >0:
    #Calculamos calor requerido
        Qcondbomba = ratio_bomba*Qreqm
        Qfueloil = ratio_caldera*Qreqm
        Qevap_bomba,PotBomba,Tent_fria,Tsal_fria,Tent_cal, Tsal_cal = f_dinamica_bomba(Pcond,PotComp_elec,Tbh,caudal_cold, caudal_hot,Qcondbomba)
        T_cal_avg = (Tent_cal+Tsal_cal)/2+273
        T_fria_avg = (Tent_fria+Tsal_fria)/2+273
        COP_carnot = T_cal_avg/(T_cal_avg - T_fria_avg)

        COP_bba= Qcondbomba/PotBomba if PotBomba !=0 else 0
        precio_kWh_UTE = calculo_precios(hora)


        if PotBomba>=0 and COP_bba < COP_carnot:
            ratio_bomba_vector.append(ratio_bomba)
            ratio_caldera_vector.append(ratio_caldera)
            costo = PotBomba*precio_kWh_UTE + ratio_caldera*Qreqm*precio_por_kWh_fueloil + Qex*precio_por_kWh_fueloil
            costos.append(costo)
            Potencia_bomba_vec.append(PotBomba)
            Qevap_bomba_vec.append(Qevap_bomba)

        ratio_bomba = ratio_bomba - 0.05
        ratio_caldera = ratio_caldera + 0.05
        ratio_bomba = round(ratio_bomba,2)
        ratio_caldera = round(ratio_caldera,2)

        if ratio_bomba == 0: # ultima iteracion del while
            PotBomba = 0
            Qcondbomba=0
            Qevap_bomba=0
            Qfueloil= Qreqm
            costo = Qfueloil*precio_por_kWh_fueloil + Qex*precio_por_kWh_fueloil
            costos.append(costo)
            Potencia_bomba_vec.append(PotBomba)
            Qevap_bomba_vec.append(Qevap_bomba)
            ratio_bomba_vector.append(ratio_bomba)
            ratio_caldera_vector.append(ratio_caldera)

        print(f'P cond= {Pcond}')
        print(f'Q cond bomba = {Qcondbomba}')
        print(f'Pot Bomba = {PotBomba}')
        print(f'ratio_bomba = {ratio_bomba}')
        print(f'costo = {costo}')
        print(f'costo vector: {costos}')

    if not costos:
        costos.append(100000)
        ratio_bomba_vector.append(1)
        ratio_caldera_vector.append(1)
        Potencia_bomba_vec.append(0)
        Qevap_bomba_vec.append(0)
        COP_bba = 0

    costofinal= min(costos)
    indicecostomin = costos.index(costofinal)
    ratio_bomba_final = (ratio_bomba_vector[indicecostomin])
    ratio_caldera_final = (ratio_caldera_vector[indicecostomin])
    Qcondbomba = ratio_bomba_final*Qreqm
    Qfueloil = ratio_caldera_final*Qreqm + Qex
    Qevap_bomba = Qevap_bomba_vec[indicecostomin]
    PotBomba = Potencia_bomba_vec[indicecostomin]

    if PotBomba==0:
        COP_bba=0
    else:
        COP_bba=Qcondbomba/PotBomba

    return PotComp_elec,PotBomba, Qfueloil,Tsal_fria,COP_bba,Qevap_bomba,Qcondbomba,costofinal



In [ ]:
def pcond_optima(Pcond, Pevap, Qevap, Tbh, caudal_cold, caudal_pozo, Tservicios,Tpozo, caudal_hot, hora):

    optimum_result = None

    def total_cost_for_pcond(Pcond):
        nonlocal optimum_result
        PotComp_elec,PotBomba, Qfueloil,Tsal_fria,COP_bba,Qevap_bomba,Qcond_bomba, costofinal = Calculo_de_potencias_de_bomba_y_compresor(Pcond, Pevap, Qevap, Tbh, caudal_cold,caudal_pozo, Tservicios, Tpozo, caudal_hot,hora)
        if costofinal!=100000:
            costo_fueloil, costo_bba, costo_comp = calculo_costos(hora,
            PotComp_elec, PotBomba, Qfueloil)
            total_cost = costo_fueloil + costo_bba + costo_comp
        else:
            total_cost= 100000000

        if optimum_result is None or total_cost < optimum_result[4]:

            optimum_result = (Pcond, costo_bba, costo_comp, costo_fueloil, total_cost, Tsal_fria,COP_bba, PotComp_elec,PotBomba,Qevap_bomba, Qcond_bomba)

        return total_cost
    
    p = 8
    costotot = []
    pfinal = []
    while p <= 15:
        costof = total_cost_for_pcond(p)
        costotot.append(costof)
        pfinal.append(p)
        p += 0.1
        p=round(p,2)
        
    df_result = pd.DataFrame([optimum_result], columns=['Pcond Optima', 'CostBomba', 'Cost Comp', 'Cost fuel oil', 'Total Cost', 'Tsal_fria calculada', 'COP calculado','PotComp calculada','PotBomba calculada', 'Q evap bomba','Qcond bomba calculada'])
    return df_result.iloc[0]

input_file_2 = r"Input_time.xlsx"
historico = pd.read_excel(input_file_2)

historico['Fecha y hora'] = pd.to_datetime(historico['Fecha y hora'])

new_columns = ['Pcond Optima', 'Cost Bomba', 'Cost Comp', 'Cost fuel oil', 'Total Cost', 'Tsal_fria calculada', 'COP calculado','PotComp calculada','PotBomba calculada','Q evap bomba', 'Qcond bomba calculada' ]
for col in new_columns:
    if col not in historico.columns:
        historico[col] = None

caudal_hot = 6.67
Tservicios = 60
Tpozo = 18

totalfilas=len(historico)
for i in range(len(historico)):
    print(i)
    avance= i/totalfilas

    formatted_percentage = f"{avance * 100:.0f}%"
    print(formatted_percentage)
    hora=time(historico['Fecha y hora'].iloc[i].hour)
    Pcond=historico['Presion de Alta'].iloc[i]
    Pevap=historico['Presion de Baja'].iloc[i]
    Qevap=historico['Qevap (kW)'].iloc[i]
    Tbh=historico['Tbh (ºC)'].iloc[i]
    caudal_pozo=historico['caudal pozo'].iloc[i]
    caudal_cold = historico['Caudal c'].iloc[i]
    row= pcond_optima(Pcond,Pevap,Qevap,Tbh,caudal_cold,caudal_pozo,Tservicios,Tpozo,caudal_hot,hora)

    for column in row.index:
        historico.at[i, column] = row[column]